# **런타임 설정**

* 딥러닝 모델 학습에 있어 **GPU**를 사용하는 것이 **CPU**에 비해 빠르기 때문에 현재 Colab의 런타임 유형을 **GPU**로 설정해줄 필요가 있음
* 상단의 **런타임** 탭 > **런타임 유형 변경** > **하드웨어 가속기**를 **GPU**로 설정

# **Google Drive 마운트**

* Colab에서 구글 드라이브 내 폴더에 접근하기 위해 마운트가 필요
* 아래와 같이 google.colab 내 drive를 import하는 것으로 마운트가 가능
* Colab 창 좌측의 폴더 탭에서도 드라이브 마운트가 가능

In [ ]:
# colab에서 드라이브 내 폴더 사용 위해 마운트
# from google.colab import drive

# drive.mount('/content/drive/')

In [ ]:
# 폴더까지의 경로
# cd '/content/drive/MyDrive/GSA_Creative_Resarch/rock_sci_paper'

In [ ]:
# 폴더 이동 확인
# ! ls

# **Requirement**

* 학습을 위해 **os, torch, torchvision** 등의 라이브러리를 활용
* **os**는 파일 등에 접근하기 위한 path 생성을 위해 활용
* **torch**는 딥러닝 학습을 위한 다양한 class와 method를 제공하기 때문에 활용
* **torchvision**는 data의 전처리를 위해 활용
* **dataset**과 **model**은 현재 폴더 내의 dataset.py와 model.py를 사용
* **random, numpy, cudnn**은 seed를 통해 랜덤성 고정을 위해 사용

In [ ]:
#필요한 라이브러리들 import
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import dataset
import model
import torch.nn.functional as F
import random
import numpy as np
import torch.backends.cudnn as cudnn

# **경로 및 파라미터 설정**

* 아래에서는 사용할 데이터 및 모델 파라미터를 저장할 **경로**를 선언
* 또한 **전체 학습 횟수나 배치 크기, 학습률, 시드** 등의 하이퍼 파라미터를 선언
* 학습 데이터 전체를 사용하여 모델을 한 번 학습시키면 1 epoch라 하여, **전체 학습 횟수(epochs)**는 학습 데이터 전체로 모델을 학습시키는 과정을 몇 회 반복할 것인지를 설정하는 값
* **배치 크기(Batch size)**는 학습 중 사용할 mini-batch의 크기를 의미하며, 모든 데이터를 한번에 사용하여 모델을 학습시키는 것은 어려우므로, 학습 중에는 몇개의 작은 mini batch로 나누어 모델을 업데이트함
* **학습률(learning rate)**은 계산된 손실 함수의 출력을 바탕으로 모델의 파라미터를 업데이터하는 정도를 의미하며, 클수록 한번 학습으로 업데이트 되는 정도가 큼
* **시드(seed)**는 코드 내 확률적 요소를 고정시켜주는 역할을 함 (ex. Array의 랜덤 셔플이 항상 같은 순서로 섞이도록 고정)
* **디바이스(device)**는 모델이 연산을 처리할 장치를 의미하며 gpu의 병렬처리 연산속도가 cpu에 비해 빠르기 때문에 사용할 수 있는 경우 gpu로 모델 연산을 처리함

In [ ]:
# 각종 path및 하이퍼 파라미터 설정
data_path = 'C:\\Users\\USER\\Desktop\\GSH_CRP\\codes\\rock_sci_paper\\data\pic128'
save_path = 'C:\\Users\\USER\\Desktop\\GSH_CRP\\codes\\rock_sci_paper\\model_para'
epochs = 50
batch_size = 16
learning_rate = 0.01
seed = 710
mode = 'lr'

In [ ]:
# seed 설정
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
cudnn.benchmark = False
cudnn.deterministic = True

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# **Transform 소개**

* **어그멘테이션(Augmentation)**은 모델에 들어가는 이미지 등의 데이터를 변화시키는 것으로, 데이터에 어그멘테이션을 주는 것으로 실제 보유한 데이터보다 더 많은 양의 데이터로 모델을 학습시키는 효과를 줄 수 있음
* **Torchvision**의 **transform**에서 해당 기능을 제공하며, 대표적인 어그멘테이션으로는 **Resize, Flip, ColorJitter, Normalize, Crop** 등이 있음
* **Resize**는 입력으로 들어오는 이미지의 크기를 동일하게 조절해주는 역할을 함
* **Flip**은 이미지를 상하좌우로 반전 시켜줌
* **ColorJitter**는 이미지의 명도, 채도, 대조 등 색상 정보를 변환시켜줌
* **Normalize**는 이미지 데이터내 값이 특정 범위 내의 값이 되도록 조절해줌
  * 서로 다른 곳에서 취득한 데이터는 여러 요인들로 인해 데이터 내 값의 범위가 크게 다를 수도 있음
  * 데이터의 특성이 다르면 모델이 오버피팅 되기 쉽고, 학습한 데이터에만 잘 작동하여 다양한 환경에 적용하기 어려움
  * 정규화(Normalization)를 통해 모델에 입력될 데이터들이 비슷한 범위가 되도록 조절하는 것으로 해당 문제를 완화할 수 있음
* **Crop**은 이미지내의 영역을 주어진 크기로 잘라 얻어냄

In [ ]:
# transform 설정
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((128,128)),
    transforms.ColorJitter(brightness=0.3),
    transforms.ColorJitter(contrast=0.3),
    transforms.ColorJitter(saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

# **데이터셋**

* 전체 데이터셋을 **Train / Validation / Test** 세종류로 나누어 사용
* **Train** 데이터셋은 학습 과정에서 모델을 학습시키고 파라미터를 업데이트 하는데 활용됨
* **Validation** 데이터셋은 학습 과정에서 모델이 제대로 학습되고 있는지, 오버피팅 되고 있지는 않은지 등 모델의 성능을 평가하기 위해 활용됨
* **Test** 데이터셋은 최종 학습된 모델의 성능을 측정하기 위해 활용됨


In [ ]:
# dataset 설정
train_dataset = dataset.RockScissorsPaper(
    transform=train_transform,
    path = data_path,
    mode = 'train'
)
val_dataset = dataset.RockScissorsPaper(
    transform=test_transform,
    path = data_path,
    mode = 'val'
)
test_dataset = dataset.RockScissorsPaper(
    transform=test_transform,
    path = data_path,
    mode = 'test'
)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

# **데이터로더**

* **데이터로더(Data Loader)**는 주어진 데이터셋으로 미니 배치를 구성하여 모델 학습 및 평가시 효율적으로 데이터를 불러오고 활용할 수 있도록 함
* 데이터셋 전체에 대해서 모델이 계산한 후 파라미터를 업데이트 할 수 있다면 효과적일테지만, 메모리나 시간 소모 측면에서 과도한 코스트를 요구함
* **미니 배치를 통한 학습**은 일반성에 대한 일부 트레이드 오프를 통해 더 적은 메모리와 시간으로 모델 학습을 가능케 하여 **효율적으로 학습**을 가능케함
* 데이터로더에는 **dataset, batch_size, shuffle** 등의 파라미터가 존재
* **dataset**은 미니 배치를 구성하려 하는 원본 데이터셋을 의미함
* **batch_size**는 구성할 미니 배치의 크기를 의미함
* **shuffle**은 data를 불러오는 순서를 섞을지 말지를 결정함


In [ ]:
# dataloader 설정
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **모델, 옵티마이저, 손실함수**

* 아래에서는 학습에 사용할 **딥러닝 모델, 손실함수, 옵티마이저**를 선언함
* **모델**을 선언한 후에는 사전에 선언한 디바이스에 올려 gpu, cpu 등에서 연산을 처리하도록 함
* **criterion**은 모델 출력과 실제 라벨과의 차이를 계산하기 위한 손실함수로 Cross Entropy Loss를 활용
* **optimize**r는 손실 함수를 통해 계산된 실제 값과의 차이(loss)를 바탕으로 모델의 파라미터를 업데이트함

In [ ]:
# 모델, 손실함수, 옵티마이저 설정
model = model.ResNet18(num_classes=3)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# **학습 코드**

* model을 학습 모드로 바꿔주기 위해 **model.train()**을 실행
* 사전에 선언한 **trainloader**를 통해 데이터와 라벨을 전달
  * batch size가 32이고 채널 수가 3, 이미지 크기가 (128, 128) 이라면 inputs의 차원은 **(32, 3, 128, 128)**
  * gpu에 올라간 모델에서 연산을 수행하기 위해서는 데이터들도 디바이스로 올려줘야함
* 매 모델 업데이트마다 optimizer를 초기화 해줘야 하기 때문에 **optimizer.zero_grad()**로 초기화 진행
* **model(inputs)**를 통해 모델에 데이터를 입력하면 모델은 각 클래스별 confidence를 예측하여 출력함
  * ex) 클래스가 총 3개라면 **(1.2, 0.6, 0.3)**과 같은 형태로 출력
  * 보통 가장 높은 confidence를 보이는 클래스를 모델의 예측값으로 활용
  * ex) 1,2,3번 클래스가 있어 모델의 출력이 **(1.2, 0.6, 0.3)**이라면, 모델이 해당 데이터를 **1**번 클래스라고 예측한 것으로 봄
* 모델의 예측값과 실제 라벨을 **criterion**에 전달하여 서로 간의 차이인 **loss를 계산**
* **loss.backward()** 를 통해 loss를 기반으로 **back propagation**을 진행
* **optimizer.step()** 을 통해 모델의 파라미터를 업데이트

In [ ]:
def train(epoch):
    print('\nEpoch: %d'%epoch)
    # model train mode로 전환
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        if mode=='lr':
            h,w = inputs.shape[-2], inputs.shape[-1]
            lr_inputs = F.interpolate(inputs, (h//8, w//8), mode='bilinear')
            lr_inputs = F.interpolate(lr_inputs, (h,w), mode='bilinear')
            outputs, _, _, _, _ = model(lr_inputs)
        else:
            outputs, _, _, _, _ = model(inputs)
            
        _, pred = torch.max(outputs, 1)
        total += outputs.size(0)
        running_acc += (pred == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    total_loss = running_loss / len(trainloader)
    total_acc = 100 * running_acc / total
    print(f'Train epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

# **평가 코드**

* **model.eval()**을 통해 model을 평가 모드로 전환
  * model에는 학습할 때와 평가할 때 다르게 동작해야 하는 레이어(ex. Drop out, Batch normalization)들이 있는데, model.train()과 model.eval()은 해당 레이어들을 on/off하는 스위치 같은 역할을 함
* **Train**에서와는 달리 **Validation, Test** 과정에서는 모델을 업데이트 하지 않기 때문에 **optimzer**는 사용하지 않음
* 또한 모델을 학습하지 않기 때문에 gradient를 계산할 필요가 없어 **torch.no_grad()**를 통해 해당 과정을 스킵하고 동작 속도를 향상시킬 수 있음
* 아래 코드에서는 평가 시 전역 변수 **BEST_SCORE**를 받아와 Validation 시 이전 성능을 넘기면 모델의 학습된 파라미터를 저장하도록 하였음
* 모델의 학습된 파라미터는 **model.state_dict()**로 접근할 수 있으며 **torch.save()**를 통해 저장할 수 있음
  * torch.save()로 모델의 state dict만을 저장할 수 있는 것은 아니며, 모델의 파라미터 저장 시 dictionary 등의 형태로 epoch, loss와 같은 정보를 같이 저장하기도 함

In [ ]:
def test(epoch, loader, test_mode='val', mode2=False):
    print('\nEpoch: %d'%epoch)
    # model eval mode로 전환
    model.eval()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    label_dict = {0:0, 1:0, 2:0}
    correct_dict = {0:0, 1:0, 2:0}
    global BEST_SCORE
    with torch.no_grad():
        for (inputs, labels) in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            if mode=='lr':
                h,w = inputs.shape[-2], inputs.shape[-1]
                lr_inputs = F.interpolate(inputs, (h//8, w//8))
                lr_inputs = F.interpolate(lr_inputs, (h,w))
                outputs, _, _, _, _ = model(lr_inputs)
            else:
                outputs, _, _, _, _ = model(inputs)

            _, pred = torch.max(outputs, 1)
            
            total += outputs.size(0)
            running_acc += (pred == labels).sum().item()
            
            if mode2:
                for i in range(len(labels)):
                    label = labels[i]
                    label_dict[label.item()] += 1
                    if (pred==labels)[i]:
                        correct_dict[label.item()] += 1
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        total_loss = running_loss / len(loader)
        total_acc = 100 * running_acc / total
        if mode2:
            print(label_dict)
            print(correct_dict)
        if total_acc >= BEST_SCORE and not test_mode=='test':
            if mode=='lr':
                path = os.path.join(save_path, f'lr_teacher.pth')
            else:
                path = os.path.join(save_path, f'teacher.pth')
            torch.save(model.state_dict(), path)
            BEST_SCORE = total_acc
        print(f'Test epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

# **모델 학습 및 평가**

* 이전에 선언된 학습 횟수 만큼 model의 학습과 Validation 데이터셋으로의 평가를 진행함
* 사전에 BEST_SCORE를 0으로 선언하여 Test 함수 내에서 접근할 수 있도록 함

In [ ]:
# 모델 학습 및 평가
BEST_SCORE = 0
for epoch in range(epochs):
    train(epoch)
    test(epoch, valloader)
    print(BEST_SCORE)

# **Test 데이터셋을 통한 평가**

* 학습 과정에서 저장해 놓았던 모델의 state dict를 **torch.load()**를 통해 로드
* 불러온 모델의 파라미터를 선언된 모델의 **load_state_dict()** 메소드를 통해 모델에 업데이트
* **Test** 데이터셋에서의 성능 평가로 모델의 최종 성능 평가 진행

In [ ]:
# 테스트셋에서 평가
if mode=='lr':
    model.load_state_dict(torch.load(os.path.join(save_path, f'lr_teacher.pth')))
else:
    model.load_state_dict(torch.load(os.path.join(save_path, f'teacher.pth')))
test(-1, testloader, 'test', True)